# Parallel Two-phase Construction


In last lecture we saw combining data structures in paralle require efficient combine operation. We saw that most of the data structure combine cannot be implemented efficiently. 

This lecture is more constructive in nature. We learn about parallel two phase data structure construction. A technique used to implement combiners efficiently.

## Two-Phase Construction

Previously we insisted that a combiner and the resulting collection have the same underlying data structure. For example, we assume that a combiner that produces an array must internally contain an array at the point when its combine method is called. In two-phase construction, the combiner has an intermediate data structure as its internal representation. This intermediate data structure is different from the resulting data structure, as we will see, and it has the following properties. 
Most data structures can be constructed in parallel using two-phase
construction.
The intermediate data structure is a data structure that:
* has an efficient combine method – $\mathcal{O}(\log n + \log m)$ or better
* has an efficient `+=` method
* can be converted to the resulting data structure in $\mathcal{O}(\frac{n}{P})$ time

Together these properties allow building the resulting data structure in two phases.

In the first phase, different processors build intermediate data structures in parallel by invoking the += method.

These intermediate data structures are then combined in a parallel reduction tree until there is a single intermediate data structure at the root. In the second phase, the result method uses the intermediate data structure to create the final data structure in parallel.

In our illustration, the final result is some array-like data structure whose subintervals are populated in parallel by different processors.

### Example: Array Combiner

Let’s implement a combiner for arrays.
Two arrays cannot be efficiently concatenated, so we will do a two-phase construction. We use `ClassTag` type as context bound to be able instantiate array.

Internally, the `ArrayCombiner` keeps the field `numElems` to store the number of elements in the combiner, and the nested `ArrayBuffer` used to store the elements. The actual elements will be stored in these entries.

We use a nested `ArrayBuffer` instead of a normal one for reasons that should soon become apparent.

We start with `+=`. This method finds the last nested array buffer in buffers and appends the element `x` to it. If the last nested `ArrayBuffer` ever gets full, it is expanded to accommodate more elements.

In [3]:
import reflect.ClassTag
import scala.collection.mutable.ArrayBuffer
class ArrayCombiner[T <: AnyRef: ClassTag](val parallelism: Int) {
private var numElems = 0
private val buffers = new ArrayBuffer[ArrayBuffer[T]]
buffers += new ArrayBuffer[T]
    
def +=(x: T) = {
buffers.last += x
numElems += 1
this
}
    
    def combine(that: ArrayCombiner[T]) = {
buffers ++= that.buffers
numElems += that.numElems
this
}
}

import reflect.ClassTag

import scala.collection.mutable.ArrayBuffer

defined class ArrayCombiner

Appending to buffer is amortized $\mathcal{O}(1)$, low constant factors – as efficient as an array buffer. If the array buffer gets full it gets expanded to accomdate more elements. 
As learned previously, appending to an array buffer takes amortized constant time. 


Next we implement the combine method. Here the reason for using nested array buffers becomes obvious. The combine method simply copies the references of the argument combiners buffers to its own buffers field. It does not need to copy the actual contents of those nested buffers, only a pointer to them. 


What is the running time of this combine? The number of computational steps is equal to the number of nested array buffers in the argument combiner. Since every array combiner is first created with only one nested array buffer, and there are exactly P array combiners created in the reduction tree, one for each processor, the buffers field will never have more than P entries. For this reason, the running of this combined method is $\mathcal{O}(P)$.

Typical desktop computers today have around four processors, and the most powerful workstations have several dozen. So P is usually negligible compared to the number of elements in the data structure, and this is still an acceptable running time for the combine operation.


Finally we can implement the result method.

Once we have the root intermediate data structure, we know the required size of the array from the `numElems` field, so we allocate the resulting array.

We then divide the array indices into chunks, pairs of starting and ending indices that each parallel task should in parallel copy. We start these tasks, wait for their completion, and then return the array. 

##  Two-Phase Construction for Arrays
Two-phase construction works for in a similar way for other data
structures. First, partition the elements, then construct parts of the final
data structure in parallel:

1. partition the indices into subintervals
2. initialize the array in parallel


## Two-Phase Construction for Hash Tables
1. partition the hash codes into buckets
2. allocate the table, and map hash codes from different buckets into different regions


## Two-Phase Construction for Search Trees

1. partition the elements into non-overlapping intervals according to their ordering
2. construct search trees in parallel, and link non-overlapping trees

## Two-Phase Construction for Spatial Data Structures
1. spatially partition the elements
2. construct non-overlapping subsets and link them

## Implementing combiners

How can we implement combiners?


1. Two-phase construction – the combiner uses an intermediate data structure with an efficient combine method to partition the elements. When result is called, the final data structure is constructed in parallel from the intermediate data structure.

2. An efficient concatenation or union operation – a preferred way when the resulting data structure allows this.

3. Concurrent data structure – different combiners share the same underlying data structure, and rely on synchronization to correctly update the data structure when += is called